**Please don't edit this cell!**

# Marks and Feedback

**Total Marks:**   XX/100

**Overall comments:**


## Part 1. Investigations into Neural Networks (35 marks)

* **Task 1**:   *Experiments with learning rate schedules* - XX/5
    * learning rate schedulers implemented
    * experiments carried out
    * further comments


* **Task 2**:   *Experiments with regularisation* - XX/5
    * L1 experiments
    * L2 experiments
    * dropout experiments
    * annealed dropout implmented
    * further experiments carried out
    * further comments
    

* **Task 3**:   *Experiments with pretraining* - XX/15
    * autoencoder pretraining implemented
    * denoising autoencoder pretraining implemented
    * CE layer-by-layer pretraining implemented
    * experiments
    * further comments


* **Task 4**:   *Experiments with data augmentation* - XX/5
    * training data augmneted using noise, rotation, ...
    * any further augmnetations
    * experiments 
    * further comments


* **Task 5**:   *State of the art* - XX/5
    * motivation for systems constructed
    * experiments
    * accuracy of best system
    * further comments



## Part 2. Convolutional Neural Networks (55 marks)

* **Task 6**:   *Implement convolutional layer* - XX/20
    * linear conv layer
    * sigmoid conv layer
    * relu conv layer
    * any checks for correctness
    * loop-based or vectorised implementations
    * timing comparisons


* **Task 7**:   *Implement maxpooling layer* - XX/10
    * implementation of non-overlapping pooling
    * generic implementation
    * any checks for correctness


* **Task 8**:   *Experiments with convolutional networks* - XX/25
    * 1 conv layer (1 fmap)
    * 1 conv layer (5 fmaps)
    * 2 conv layers
    * further experiments



## Presentation (10 marks)

* ** Marks:**   XX/10
    * Concise description of each system constructed
    * Experiment design and motivations for different systems
    * Presentation of results - graphs, tables, diagrams
    * Conclusions



# Coursework #2

## Introduction


## Previous Tutorials

Before starting this coursework make sure that you have completed the following labs:

* [04_Regularisation.ipynb](https://github.com/CSTR-Edinburgh/mlpractical/blob/master/04_Regularisation.ipynb) - regularising the model
* [05_Transfer_functions.ipynb](https://github.com/CSTR-Edinburgh/mlpractical/blob/master/05_Transfer_functions.ipynb) - building and training different activation functions
* [06_MLP_Coursework2_Introduction.ipynb](https://github.com/CSTR-Edinburgh/mlpractical/blob/master/06_MLP_Coursework2_Introduction.ipynb) - Notes on numpy and tensors


## Submission
**Submission Deadline:  Thursday 14 January 2016, 16:00** 

Submit the coursework as an ipython notebook file, using the `submit` command in the terminal on a DICE machine. If your file is `06_MLP_Coursework1.ipynb` then you would enter:

`submit mlp 2 06_MLP_Coursework1.ipynb` 

where `mlp 2` indicates this is the second coursework of MLP.

After submitting, you should receive an email of acknowledgment from the system confirming that your submission has been received successfully. Keep the email as evidence of your coursework submission.

**Please make sure you submit a single `ipynb` file (and nothing else)!**

**Submission Deadline:  Thursday 14 January 2016, 16:00** 



## Getting Started
Please enter your student number and the date in the next code cell.

In [ ]:
#MLP Coursework 2
#Student number: s1210107
#Date: 16/12/2015

# Part 1. Investigations into Neural Networks (35 marks)

In this part you are may choose exactly what you implement. However, you are expected to express your motivations, observations, and findings in a clear and cohesive way. Try to make it clear why you decided to do certain things. Use graphs and/or tables of results to show trends and other characteristics you think are important. 

For example, in Task 1 you could experiment with different schedulers in order to compare their convergence properties. In Task 2 you could look into (and visualise) what happens to weights when applying L1 and/or L2 regularisation when training. For instance, you could create sorted histograms of weight magnitudes in in each layer, etc..

**Before submission, please collapse all the log entries into smaller boxes (by clicking on the bar on the left hand side)**

### Task 1 - Experiments with learning rate schedules (5 marks)

Investigate the effect of learning rate schedules on training and accuracy.  Implement at least one additional learning rate scheduler mentioned in the lectures. 

In [2]:
#load the corresponding code here, and also attach scripts that run the experiments ()

import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

training_set_size = train_dp.batch_size * train_dp._max_num_batches

#Baseline experiment

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed, LearningRateExponential, LearningRateReciprocal

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
learning_rate = 0.5
max_epochs = 30
cost = CECost()

lr_fixed = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
lr_exponential = LearningRateExponential(learning_rate=learning_rate, max_epochs=max_epochs, train_set_size=training_set_size)
lr_reciprocal = LearningRateReciprocal(learning_rate=learning_rate, max_epochs=max_epochs, train_set_size=training_set_size)

lr_schedulers = [lr_fixed,lr_exponential,lr_reciprocal]


stats = []
for lr_scheduler in lr_schedulers:

    layer = 1
    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    for i in xrange(1, layer):
        logger.info("Stacking hidden layer (%s)" % str(i+1))
        model.add_layer(Sigmoid(idim=nhid, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))

    # define the optimiser, here stochasitc gradient descent
    # with fixed learning rate and max_epochs
    lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

INFO:root:Initialising data providers...
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.624. Accuracy is 8.60%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 3.053. Accuracy is 58.00%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.600. Accuracy is 81.41%
INFO:mlp.optimisers:Epoch 1: Took 1 seconds. Training speed 1100 pps. Validation speed 17859 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.518. Accuracy is 83.50%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.497. Accuracy is 85.12%
INFO:mlp.optimisers:Epoch 2: Took 1 seconds. Training speed 1112 pps. Validation speed 17546 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.324. Accuracy is 91.00%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.458. Accuracy is 85.38%
INFO:mlp.optimisers:Epoch 3: Took 1 seconds. Training speed 1100 pps. Valid

In [41]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy

# arranging the data for plotting
test_results = [stat[2] for stat in stats]
test_ce_costs = [test_result[0] for test_result in test_results]
test_accuracies = [test_result[1] for test_result in test_results]

fixed_training_ce_costs = [test_result[1] for test_result in stats[0][1]]
exponential_training_ce_costs = [test_result[1] for test_result in stats[1][1]]
reciprocal_training_ce_costs = [test_result[1] for test_result in stats[2][1]]

fixed_training_accuracies = [test_result[1] for test_result in stats[0][1]]
exponential_training_accuracies = [test_result[1] for test_result in stats[1][1]]
reciprocal_training_accuracies = [test_result[1] for test_result in stats[2][1]]

epochs = xrange(0,31)
names = ['fixed','exponential','reciprocal']

# training plot
fig = plt.figure()

plt.plot(fixed_training_accuracies, epochs, label="fixed")
plt.plot(exponential_training_accuracies, epochs, label="exponential")
plt.plot(reciprocal_training_accuracies, epochs, label="reciprocal")

plt.xlabel('Epoch number')
plt.ylabel('Training accuracy (%)')
plt.title('Fixed, exponential, and reciprocal learning rate scheduler test accuracy')
plt.legend(loc='left')

plt.grid(True)
plt.savefig("pictures/test.png")


fig = plt.figure()

width = .25
ind = numpy.arange(len(test_accuracies))
plt.bar(ind, test_accuracies)
plt.xticks(ind + width / 2, names)

plt.xlabel('Different learning rate schedulers')
plt.ylabel('Accuracy (%)')
plt.title('Fixed, exponential, and reciprocal learning rate scheduler test accuracy')
plt.grid(True)
plt.savefig("pictures/test.png")
plt.show()

### Task 2 - Experiments with regularisers (5 marks)

Investigate the effect of different regularisation approaches (L1, L2, dropout).  Implement the annealing dropout scheduler (mentioned in lecture 5). Do some further investigations and experiments with model structures (and regularisers) of your choice. 

In [2]:
import numpy
import logging
from mlp.dataset import MNISTDataProvider

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser
from mlp.dataset import MNISTDataProvider #import data provider
from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed, DropoutFixed, DropoutAnnealed

#load the corresponding code here, and also attach scripts that run the experiments ()

logger = logging.getLogger()
logger.setLevel(logging.INFO)

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed, LearningRateExponential, LearningRateReciprocal

rng = numpy.random.RandomState([2015,10,10])

logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

#some hyper-parameters
nhid = 800
learning_rate = 0.5
max_epochs = 30

cost = CECost()
    
stats = []
layer = 1

# L1 parameters
l2_weight = 0

for l1_weight in [0.0001, 0.001, 0.01, 0.1]:

    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    for i in xrange(1, layer):
        logger.info("Stacking hidden layer (%s)" % str(i+1))
        model.add_layer(Sigmoid(idim=nhid, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))

    # define the optimiser, here stochasitc gradient descent
    # with fixed learning rate and max_epochs
    lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    dp_scheduler = DropoutFixed(0.5, 0.5)
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler,
                             dp_scheduler=None,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))
    


INFO:root:Initialising data providers...
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 65.717. Accuracy is 8.60%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 65.646. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 63.716. Accuracy is 57.50%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 61.280. Accuracy is 80.27%
INFO:mlp.optimisers:Epoch 1: Took 2 seconds. Training speed 705 pps. Validation speed 16668 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 58.420. Accuracy is 82.70%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 58.393. Accuracy is 84.29%
INFO:mlp.optimisers:Epoch 2: Took 2 seconds. Training speed 695 pps. Validation speed 16951 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 55.427. Accuracy is 90.50%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 55.567. Accuracy is 84.00%
INFO:mlp.optimisers:Epoch 3: Took 2 seconds. Training speed 700 pps. 

In [ ]:
# L2 parameters

l1_weight = 0

for l2_weight in [0.0001, 0.001, 0.01, 0.1]:

    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    for i in xrange(1, layer):
        logger.info("Stacking hidden layer (%s)" % str(i+1))
        model.add_layer(Sigmoid(idim=nhid, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))

    # define the optimiser, here stochasitc gradient descent
    # with fixed learning rate and max_epochs
    lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    dp_scheduler = DropoutFixed(0.5, 0.5)
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler,
                             dp_scheduler=None,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))



INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 6.791. Accuracy is 8.30%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 6.773. Accuracy is 10.28%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 7.076. Accuracy is 60.70%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 4.872. Accuracy is 81.40%
INFO:mlp.optimisers:Epoch 1: Took 2 seconds. Training speed 981 pps. Validation speed 17859 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 4.739. Accuracy is 85.70%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 4.742. Accuracy is 84.68%
INFO:mlp.optimisers:Epoch 2: Took 2 seconds. Training speed 981 pps. Validation speed 17546 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 4.562. Accuracy is 89.80%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 4.622. Accuracy is 88.42%
INFO:mlp.optimisers:Epoch 3: Took 2 seconds. Training speed 981 pps. Validation speed 16395 pps.
INFO:mlp.optimisers:

In [ ]:
# annealed and fixed dropout

dp_fixed_scheduler = DropoutFixed(0.5, 0.5)
dp_annealed_scheduler = DropoutAnnealed(0.5, 0.5)
dp_schedulers = [dp_fixed_scheduler, dp_annealed_scheduler]
l1_weight = 0.0
l2_weight = 0.0    
    
for dp_scheduler in dp_schedulers:

    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    for i in xrange(1, layer):
        logger.info("Stacking hidden layer (%s)" % str(i+1))
        model.add_layer(Sigmoid(idim=nhid, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))

    # define the optimiser, here stochasitc gradient descent
    # with fixed learning rate and max_epochs
    lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler,
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy

# arranging the data for plotting
test_results = [stat[2] for stat in stats]
test_ce_costs = [test_result[0] for test_result in test_results]
test_accuracies = [test_result[1] for test_result in test_results]

fixed_training_ce_costs = [test_result[1] for test_result in stats[0][1]]
exponential_training_ce_costs = [test_result[1] for test_result in stats[1][1]]
reciprocal_training_ce_costs = [test_result[1] for test_result in stats[2][1]]

fixed_training_accuracies = [test_result[1] for test_result in stats[0][1]]
exponential_training_accuracies = [test_result[1] for test_result in stats[1][1]]
reciprocal_training_accuracies = [test_result[1] for test_result in stats[2][1]]

epochs = xrange(0,31)
names = ['fixed','exponential','reciprocal']

# training plot
fig = plt.figure()

plt.plot(fixed_training_accuracies, epochs, label="fixed")
plt.plot(exponential_training_accuracies, epochs, label="exponential")
plt.plot(reciprocal_training_accuracies, epochs, label="reciprocal")

plt.xlabel('Epoch number')
plt.ylabel('Training accuracy (%)')
plt.title('Fixed, exponential, and reciprocal learning rate scheduler test accuracy')
plt.legend(loc='left')

plt.grid(True)
plt.savefig("pictures/test.png")


fig = plt.figure()

width = .25
ind = numpy.arange(len(test_accuracies))
plt.bar(ind, test_accuracies)
plt.xticks(ind + width / 2, names)

plt.xlabel('Different learning rate schedulers')
plt.ylabel('Accuracy (%)')
plt.title('Fixed, exponential, and reciprocal learning rate scheduler test accuracy')
plt.grid(True)
plt.savefig("pictures/test.png")
plt.show()

### Task 3 - Experiments with pretraining (15 marks)

Implement pretraining of multi-layer networks with autoencoders, denoising autoencoders, and using  layer-by-layer cross-entropy training.  

Implementation tip: You could add the corresponding methods to `optimiser`, namely, `pretrain()` and `pretrain_epoch()`, for autoencoders. Simiilarly, `pretrain_discriminative()` and `pretrain_epoch_discriminative()` for cross-entropy layer-by-layer pretraining. Of course, you can modify any other necessary pieces, but include all the modified fragments below.

### Task 4 - Experiments with data augmentation (5 marks)

Using the standard MNIST training data, generate some augmented training examples (for example, using noise or rotation). Perform experiments on using this expanded training data.

### Task 5 - State of the art (5 marks)

Using any techniques you have learnt so far (combining any number of them), build and train the best model you can (no other constraints).

# Part 2. Convolutional Neural Networks (55 marks)

In this part of the coursework, you are required to implement deep convolutional networks.  This includes code for forward prop, back prop, and weight updates for convolutional and max-pooling layers, and should support the stacking of convolutional + pooling layers.  You should implement all the parts relating to the convolutional layer in the mlp/conv.py module; if you decide to implement some routines in cython, keep them in mlp/conv.pyx). Attach both files in this notebook.

Implementation tips: Look at [lecture 7](http://www.inf.ed.ac.uk/teaching/courses/mlp/2015/mlp07-cnn.pdf) and [lecture 8](http://www.inf.ed.ac.uk/teaching/courses/mlp/2015/mlp08-cnn2.pdf), and the introductory tutorial, [06_MLP_Coursework2_Introduction.ipynb](https://github.com/CSTR-Edinburgh/mlpractical/blob/master/06_MLP_Coursework2_Introduction.ipynb)

### Task 6 -  Implement convolutional layer (20 marks)

Implement linear convolutional layer, and then extend to sigmoid and ReLU transfer functions (do it in a similar way to fully-connected layers). Include all relevant code.  It is recommended that you first implement in the naive way with nested loops (python and/or cython);  optionally you may then implement in a vectorised way in numpy.  Include logs for each way you implement the convolutional layer, as timings for different implementations are of interest.  Include all relevant code.

### Task 7 - Implement max-pooling layer (10 marks)

Implement a max-pooling layer. Non-overlapping pooling (which was assumed in the lecture presentation) is required. You may also implement a more generic solution with striding as well. Include all relevant code.

### Task 8 - Experiments with convolutional networks (25 marks)

Construct convolutional networks with a softmax output layer and a single fully connected hidden layer. Your first experiments should use one convolutional+pooling layer.  As a default use convolutional kernels of dimension 5x5 (stride 1) and pooling regions of 2x2 (stride 2, hence non-overlapping).

*  Implement and test a convolutional network with 1 feature map
*  Implement and test a convolutional network with 5 feature maps

Explore convolutional networks with two convolutional layers, by implementing, training, and evaluating a network with two convolutional+maxpooling layers with 5 feature maps in the first convolutional layer,  and 10 feature maps in the second convolutional layer.

Carry out further experiments to optimise the convolutional network architecture (you could explore kernel sizes and strides, number of feature maps, sizes and strides of pooling operator, etc. - it is up to you).

**This is the end of coursework 2.**

Please remember to save your notebook, and submit your notebook following the instructions at the top.  Please make sure that you have executed all the code cells when you submit the notebook.
